# Quant Finance - Clase 7 - A Quant Library

## Libreria Stock Options

Modulos necesarios

varios de library (en github),

matplotlib,

datetime,

sys

### Importo los modulos

In [ ]:
import sys
sys.path.append('..')
#Para poder leer los modulos dentro del directorio 
from library.stock import Stock
from library.options import StockOption
from library.market_data import MarketData
from library.curves import FlatZeroCurve
from library.volatility_surfaces import FlatVolatilitySurface
from library.bspricer import BlackScholesFormulaPricer
from library.mcpricer import SimpleMonteCarloPricer
from library.bsmodel import BlackScholesModel
from library.year_fractions import Actual360
import yfinance as yf
import datetime

#### 1) Fecha de valuación

In [ ]:
t = datetime.date(2020,6,12)

#### 2) Creamos el stock y agregamos solo su precio de cierre en t

In [ ]:
stock = Stock( name = 'test_stock' )
stock.add_close_price( time = t, price = 42 )

#### 3) Creamos la superficie de volatilidad y agregamos volatilidad al cierre de t

In [ ]:
vol_surface = FlatVolatilitySurface( name = 'test_vol_surface' )
vol_surface.add_close_volatility( time = t, strike = None, maturity = None, volatility = 0.2 )

#### 4) Creamos una curva de descuento flat y agregamos la tasa de descuento al cierre de t

In [ ]:
zero_curve = FlatZeroCurve( name = 'test_zero_curve', year_fraction = Actual360 )
zero_curve.add_close_zero_rate( time = t, zero_rate = 0.1 )

#### 5) Creamos el objeto que contiene la información del mercado

In [ ]:
market_data = MarketData()
market_data.add_stock( stock )
market_data.add_volatility_surface( vol_surface )
market_data.add_zero_curve( zero_curve )

#### 6) Definimos el instrumento de derivado (stock)

In [ ]:
maturity = datetime.date(2020,12,9)
option_type = 'call'
option = StockOption(   underlier_name = 'test_stock', 
                        strike = 40, 
                        maturity = maturity, 
                        payoff_name = option_type
                    )

#### 7) Calculamos el precio usando diferentes pricers

#### A) Formula de Black-Scholes

In [ ]:
pricer = BlackScholesFormulaPricer( option = option, market_data = market_data ) 
pricer_parameters = {'underlier_vol': 'test_vol_surface', 'zero_curve': 'test_zero_curve'}
pricer.initialize_pricer(pricer_parameters)
print( 'BS formula:' )
print( pricer.npv(time = t) )

#### B) Monte-Carlo

In [ ]:
pricer = SimpleMonteCarloPricer( option = option, market_data = market_data )
model = BlackScholesModel( rate = 0.1, volatility = 0.2 )
pricer_parameters = { 'zero_curve': 'test_zero_curve', 'samples': 1000000 }
pricer.initialize_model(model)    
pricer.initialize_pricer(pricer_parameters)    
print( 'Simple Montecarlo:' )
print( pricer.npv(time = t) )